In [1]:
import os
import pandas as pd
import sys
import time

from PIL import Image
from sqlalchemy import create_engine, func
from sqlalchemy.orm import sessionmaker

sys.path.append('../..')
sys.path.append('../../../ajna_docs/commons')


In [2]:
from ajna_commons.flask.conf import SQL_URI
from ajna_commons.utils.images import mongo_image
from virasana.db import mongodb as db


2021-03-05 11:02:01,515 ajna         INFO     Configuração de log efetuada


Fazendo log de erros e alertas no arquivo  ../../../ajna_docs/commons\ajna_commons\flask\error.log
Fazendo log de atividade no arquivo  ../../../ajna_docs/commons\ajna_commons\flask\access.log


In [23]:
%load_ext autoreload 
%autoreload 2
from virasana.integracao.conformidade_alchemy import Conformidade
from virasana.scripts.conformidadeupdate import completa_conformidade, preenche_isocode, preenche_bbox
engine = create_engine(SQL_URI)
# completa_conformidade(db, engine, 200)
# preenche_isocode(db, engine, 200)
preenche_bbox(db, engine, 20)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
2
2
1
2
1
2
2
1
1
1
2
1
1
1
2
1
1
1
1
2


2021-03-05 11:22:27,353 ajna         INFO     20 bbox preenchidos em 5.685836315155029 segundos.0.2842918157577515 por registro


# Checagens de conformidade

In [6]:
Session = sessionmaker(bind=engine)
session = Session()

In [7]:
rs = session.execute('SELECT count(*) FROM dbmercante.ajna_conformidade;')

In [8]:
rs.scalar()

124896

In [19]:
SQL_CONFORMIDADE = '''
SELECT cod_recinto as Recinto, count(*) as "Qtde de imagens",
 avg(height) as "Linhas",
 avg(width) as "Colunas",
 avg(ratio) as "Relação largura/altura",
 SUM(if(height < 700, 1, 0)) / count(*) * 100 as "% Tamanho pequeno",
 SUM(if(ratio < 1.9, 1, 0)) / count(ratio) * 100 as "% relação abaixo 1.9",
 avg(laplacian) as "Índice de nitidez ou ruído"
 from ajna_conformidade 
 WHERE dataescaneamento BETWEEN "2021-02-22" AND "2021-03-01"
 group by cod_recinto
 '''
df = pd.read_sql(SQL_CONFORMIDADE, engine)

In [20]:
df

,Recinto,Qtde de imagens,Linhas,Colunas,Relação largura/altura,% Tamanho pequeno,% relação abaixo 1.9,Índice de nitidez ou ruído
0,BANDEIRANTES,388,799.3557,1956.3531,2.447706,0.0000,20.3608,113.5556
1,BTP1,5748,799.0303,2102.6667,2.631571,0.0000,5.5150,91.0452
2,BTP2,2450,799.0265,1858.0176,2.325355,0.0000,16.9796,112.6529
3,DEICMAR,727,799.0344,1351.1568,1.690922,0.0000,80.7428,131.3846
4,ECOPORTO,1188,798.9848,2125.6490,2.660362,0.0000,3.6195,85.4909
5,EMBRAPORT,3350,799.3319,3809.2457,4.766209,0.0000,0.1493,864.7500
6,EUDMARCO,445,287.4966,1022.6629,3.835888,83.8202,0.0000,1359.0000
7,LOCALFRIO,1286,799.0008,1753.0101,2.194456,0.0000,4.6656,112.8261
8,MARIMEX,1158,799.0294,1945.5181,2.434940,0.0000,12.5216,80.4545
9,NOVALOG,543,798.9448,1728.9337,2.164236,0.0000,12.7072,492.6667
